# 베스트셀러 인터랙티브 분석 (Plotly)
이 노트북은 기존의 정적 시각화를 개선하여 더 효과적으로 인사이트를 탐색할 수 있는 대시보드들을 제공합니다.

In [3]:
import os
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from dotenv import load_dotenv
from supabase import create_client

load_dotenv()
supabase = create_client(os.getenv('SUPABASE_URL'), os.getenv('SUPABASE_KEY'))

# 데이터 로드
bestsellers_res = supabase.table('bestsellers').select('*').execute()
df_bestsellers = pd.DataFrame(bestsellers_res.data)

books_res = supabase.table('books').select('product_code, title, category_1, category_2, category_3').execute()
df_books = pd.DataFrame(books_res.data)

# 데이터 병합 및 전처리
df = df_bestsellers.merge(df_books, on='product_code', how='left')
df['bestseller_month'] = pd.to_datetime(df['bestseller_month'])
df['month_str'] = df['bestseller_month'].dt.strftime('%Y-%m')
df = df.sort_values(['bestseller_month', 'rank'])

## 1. 월별 카테고리 비중 변화 (Stacked Area Chart)
시간의 흐름에 따라 어떤 카테고리가 강세를 보이는지 한눈에 파악할 수 있습니다.

In [4]:
monthly_cat = df.groupby(['month_str', 'category_1']).size().reset_index(name='count')
fig1 = px.area(monthly_cat, x='month_str', y='count', color='category_1', 
               title='월별 카테고리 점유율 변화',
               labels={'month_str': '월', 'count': '도서 수', 'category_1': '카테고리'},
               line_group='category_1')
fig1.show()

## 2. 주요 도서의 순위 변동 추이 (Bump Chart)
베스트셀러 상위권 도서들이 매달 어떻게 순위가 바뀌는지 추적합니다.

In [5]:
# 여러 번 차트에 등장한 책들 위주로 필터링
top_books = df['title'].value_counts().head(15).index
df_top = df[df['title'].isin(top_books)]

fig2 = px.line(df_top, x='month_str', y='rank', color='title', markers=True,
              title='상위 인기 도서 순위 변동 추이',
              labels={'month_str': '월', 'rank': '순위', 'title': '도서명'})
fig2.update_yaxes(autorange='reversed') # 1위가 위에 오도록 설정
fig2.show()

## 3. 리뷰 수 vs 평점 상관관계 (Bubble Chart)
리뷰가 많은 책이 평점도 높은지, 카테고리별 특성은 어떤지 분석합니다.

In [6]:
fig3 = px.scatter(df.drop_duplicates(subset=['product_code']), 
                 x='review_count', y='rating', color='category_1', 
                 size='review_count', hover_name='title',
                 title='도서별 리뷰 수와 평점의 관계',
                 labels={'review_count': '리뷰 수', 'rating': '평점', 'category_1': '카테고리'})
fig3.show()

## 4. 월별 카테고리 평균 평점 (Heatmap)
매달 독자 만족도가 높은 카테고리가 어디인지 한눈에 확인합니다.

In [7]:
avg_rating = df.groupby(['month_str', 'category_1'])['rating'].mean().unstack()
fig4 = px.imshow(avg_rating, 
                labels=dict(x='카테고리', y='월', color='평균 평점'),
                title='월별 카테고리별 평균 평점 흐름')
fig4.show()

## 5. 카테고리 분포 (Tree Map)
전체 베스트셀러 내에서 각 카테고리가 차지하는 비중을 계층적으로 보여줍니다.

In [8]:
fig5 = px.treemap(df, path=['category_1', 'title'], 
                  title='카테고리 및 도서 비중 트리맵',
                  labels={'count': '노출 횟수'})
fig5.show()

ValueError: ('None entries cannot have not-None children', ('거시경제/금융정책', '돈의 흐름은 되풀이된다'))